In [1]:
import tensorflow as tf
from keras.preprocessing.image import ImageDataGenerator
tf.__version__

'2.11.0'

In [2]:
# Image Data Generator
train_datagen = ImageDataGenerator(
    rescale=1./255,
    shear_range=0.2,
    zoom_range=0.2,
)

test_datagen = ImageDataGenerator(rescale=1./255)

batchSize = 16
size = 224

training_set = train_datagen.flow_from_directory(
    "data/train",
    target_size=(size, size),
    batch_size=batchSize,
    class_mode="categorical"
)

validation_set = test_datagen.flow_from_directory(
    "data/validation",
    target_size=(size, size),
    batch_size=batchSize,
    class_mode="categorical"
)

test_set = test_datagen.flow_from_directory(
    "data/test",
    target_size=(size, size),
    batch_size=batchSize,
    class_mode="categorical",
    shuffle=False
)

Found 168 images belonging to 3 classes.
Found 36 images belonging to 3 classes.
Found 36 images belonging to 3 classes.


In [3]:
import tensorflow as tf
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam

In [4]:
base_model = MobileNetV2(weights='imagenet', include_top=False, input_shape=(size, size, 3))

In [5]:
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(3, activation='softmax')(x)

model = Model(inputs=base_model.input, outputs=x)

for layer in base_model.layers:
    layer.trainable = False

In [ ]:
import tensorflow as tf
from tensorflow.keras.applications import MobileNet

#size = 128

base_model = MobileNet(input_shape=(size, size, 3), weights='imagenet', include_top=False, alpha=0.5)

x = base_model.output
x = tf.keras.layers.GlobalAveragePooling2D()(x)
x = tf.keras.layers.Dense(3, activation='softmax')(x)

smaller_model = tf.keras.Model(inputs=base_model.input, outputs=x)

In [6]:
model.compile(optimizer=Adam(learning_rate=0.001), loss='categorical_crossentropy', metrics=['accuracy'])

In [7]:
epochs = 20

history = model.fit(
    training_set,
    epochs=epochs,
    validation_data=validation_set,
    steps_per_epoch=len(training_set),
    validation_steps=len(validation_set)
)

test_loss, test_accuracy = model.evaluate(test_set)
print("Test Loss: {:.4f}, Test Accuracy: {:.4f}".format(test_loss, test_accuracy))

Epoch 1/20
11/11 [==============================] - 6s 362ms/step - loss: 0.8044 - accuracy: 0.6190 - val_loss: 0.5868 - val_accuracy: 0.6944
Epoch 2/20
11/11 [==============================] - 3s 273ms/step - loss: 0.5081 - accuracy: 0.8214 - val_loss: 0.5005 - val_accuracy: 0.8333
Epoch 3/20
11/11 [==============================] - 3s 274ms/step - loss: 0.4220 - accuracy: 0.7619 - val_loss: 0.4222 - val_accuracy: 0.8056
Epoch 4/20
11/11 [==============================] - 3s 276ms/step - loss: 0.3613 - accuracy: 0.8333 - val_loss: 0.3792 - val_accuracy: 0.8889
Epoch 5/20
11/11 [==============================] - 3s 271ms/step - loss: 0.3079 - accuracy: 0.8988 - val_loss: 0.3485 - val_accuracy: 0.8889
Epoch 6/20
11/11 [==============================] - 3s 285ms/step - loss: 0.2685 - accuracy: 0.9226 - val_loss: 0.3203 - val_accuracy: 0.8889
Epoch 7/20
11/11 [==============================] - 3s 282ms/step - loss: 0.2822 - accuracy: 0.8690 - val_loss: 0.3377 - val_accuracy: 0.8611
Epoch 

In [ ]:
# Save the Keras model as an H5 file
model.save("trained_model.h5")

In [ ]:
def representative_dataset_gen():
    for _ in range(100):  # You can adjust the number of samples used here
        # Get a single batch of data
        x, _ = next(validation_set)
        # Yield the data as a 1-item list containing a numpy array
        yield [x.astype("float32")]


In [ ]:
import tensorflow as tf

# Load the trained Keras model
model = tf.keras.models.load_model("trained_mobilenetv1_128x128.h5")

# Set up the converter
converter = tf.lite.TFLiteConverter.from_keras_model(model)
converter.optimizations = [tf.lite.Optimize.DEFAULT]
converter.target_spec.supported_ops = [tf.lite.OpsSet.TFLITE_BUILTINS_INT8]
converter.inference_input_type = tf.int8
converter.inference_output_type = tf.int8
converter.representative_dataset = representative_dataset_gen

# Perform quantization
tflite_quantized_model = converter.convert()

# Save the quantized TensorFlow Lite model
with open("trained_model_quantized.tflite", "wb") as f:
    f.write(tflite_quantized_model)
